In [107]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StringType,StructField,StructType,IntegerType


In [108]:
spark=SparkSession.builder.master("local").appName("sql_df").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

In [109]:
schema= StructType([StructField("id",IntegerType(),True),StructField("names",StringType(),True)])
names=spark.read.schema(schema).option("sep"," ").csv(r"C:\Spark Course\SuperHeroes\MarvelNames.txt")
names

id,names
1,24-HOUR MAN/EMMANUEL
2,3-D MAN/CHARLES CHAN
3,4-D MAN/MERCURIO
4,8-BALL/
5,A
6,A'YIN
7,"ABBOTT, JACK"
8,ABCISSA
9,ABEL
10,ABOMINATION/EMIL BLO


In [110]:
ids=spark.read.text(r"C:\Spark Course\SuperHeroes\MarvelGraph.txt")
ids

value
5988 748 1722 375...
5989 4080 4264 44...
5982 217 595 1194...
5983 1165 3836 43...
5980 2731 3712 15...
5981 3569 5353 40...
5986 2658 3712 26...
5987 2614 5716 17...
5984 590 4898 745...
5985 3233 2254 21...


In [111]:
newdf=ids.withColumn("id",func.split(func.col("value")," ")[0]).withColumn("total_count",func.size(func.split(func.col("value")," "))-1)

final = newdf.select("id","total_count").groupBy("id").agg(func.sum("total_count")).sort(func.col("sum(total_count)").desc()).first()



In [112]:
name_final=names.filter(func.col("id") == int(final[0]))
name_final

id,names
859,CAPTAIN AMERICA


In [115]:
df1=newdf.select("id","total_count").groupBy("id").agg(func.sum("total_count")).sort(func.col("sum(total_count)").desc())
df1 = df1.filter(func.col("sum(total_count)")==1)
df1

id,sum(total_count)
467,1
577,1
3490,1
3489,1
2139,1
1089,1
1841,1
4517,1
5028,1
835,1


In [116]:
x=names.join(df1,"id")
x

id,names,sum(total_count)
467,BERSERKER II,1
577,BLARE/,1
3490,MARVEL BOY II/MARTIN,1
3489,MARVEL BOY/MARTIN BU,1
2139,"GIURESCU, RADU",1
1089,CLUMSY FOULUP,1
1841,FENRIS,1
4517,RANDAK,1
5028,SHARKSKIN,1
835,"CALLAHAN, DANNY",1
